In [10]:
from annoy import AnnoyIndex
import numpy as np
import pandas as pd
import re
import os
from dotenv import load_dotenv
load_dotenv()
import cohere
co = cohere.Client(os.getenv("COHERE_API_KEY"))

from rich import print
from rich import pretty
pretty.install()


import weaviate
import pandas as pd
import numpy as np
from icecream import ic
from utils import dense_retrieval, print_result, keyword_search
from rich import print


In [3]:
# auth_config = weaviate.auth.AuthApiKey(
#     api_key=os.environ['WEAVIATE_API_KEY'])
# client = weaviate.Client(
#     url=os.environ['WEAVIATE_API_URL'],
#     auth_client_secret=auth_config,
#     additional_headers={
#         "X-Cohere-Api-Key": os.environ['COHERE_API_KEY'],
#     }
# )

client = weaviate.Client(
    auth_client_secret = weaviate.AuthApiKey(api_key = os.getenv("WEAVIATE_API_KEY")),

    # 10 million records from 10 different languages. One million records for each language.
    # Each row is a paragraph from wikipedia
    url = "https://cohere-demo.weaviate.network/",
    additional_headers={
        "X-Cohere-Api-Key": os.getenv("COHERE_API_KEY"),

    }
)


In [2]:
text = """
Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.
It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.
Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.

Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.
Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.
Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.
Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.
Interstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.

Interstellar premiered on October 26, 2014, in Los Angeles.
In the United States, it was first released on film stock, expanding to venues using digital projectors.
The film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.
It received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.
It has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.
Interstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades"""


# Chunking

In [3]:
# # Split into a list of sentences
# texts = text.split('.')

# # Clean up to remove empty spaces and new lines
# texts = np.array([t.strip(' \n') for t in texts])
# texts


array(['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan',
       'It stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine',
       'Set in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind',
       'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007',
       'Caltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar',
       'Cinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm',
       'Principal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles',

In [7]:
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts])
print(texts)


['Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.'
 'Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical and miniature

In [8]:
title = 'Interstellar (film)'

texts = np.array([f"{title} {t}" for t in texts])
print(texts)


['Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.'
 'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstella

# Now lets Embed the paragraph

In [11]:
response = co.embed(texts = texts.tolist()).embeddings
embeds = np.array(response)
embeds.shape


default model on embed will be deprecated in the future, please specify a model in the request.


(3, 4096)

# Create search index with annoy

In [13]:
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])

search_index.build(10) # 10 trees
search_index.save('test.ann')


True

In [14]:
pd.set_option('display.max_colwidth', None)

def search(query):

  # Get the query's embedding
  query_embed = co.embed(texts=[query]).embeddings

  # Retrieve the nearest neighbors
  similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    3,
                                                  include_distances=True)
  # Format the results
  results = pd.DataFrame(data={'texts': texts[similar_item_ids[0]],
                              'distance': similar_item_ids[1]})

  print(texts[similar_item_ids[0]])

  return results


In [15]:
query = "How much did the film make?"
search(query)


default model on embed will be deprecated in the future, please specify a model in the request.


['Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles.\nIn the United States, it was first released on film stock, expanding to venues using digital projectors.\nThe film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.\nIt received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.\nIt has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.\nInterstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades'
 'Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins 

,texts,distance
0,"Interstellar (film) Interstellar premiered on October 26, 2014, in Los Angeles.\nIn the United States, it was first released on film stock, expanding to venues using digital projectors.\nThe film had a worldwide gross over $677 million (and $773 million with subsequent re-releases), making it the tenth-highest grossing film of 2014.\nIt received acclaim for its performances, direction, screenplay, musical score, visual effects, ambition, themes, and emotional weight.\nIt has also received praise from many astronomers for its scientific accuracy and portrayal of theoretical astrophysics. Since its premiere, Interstellar gained a cult following,[5] and now is regarded by many sci-fi experts as one of the best science-fiction films of all time.\nInterstellar was nominated for five awards at the 87th Academy Awards, winning Best Visual Effects, and received numerous other accolades",1.092562
1,"Interstellar (film) Brothers Christopher and Jonathan Nolan wrote the screenplay, which had its origins in a script Jonathan developed in 2007.\nCaltech theoretical physicist and 2017 Nobel laureate in Physics[4] Kip Thorne was an executive producer, acted as a scientific consultant, and wrote a tie-in book, The Science of Interstellar.\nCinematographer Hoyte van Hoytema shot it on 35 mm movie film in the Panavision anamorphic format and IMAX 70 mm.\nPrincipal photography began in late 2013 and took place in Alberta, Iceland, and Los Angeles.\nInterstellar uses extensive practical and miniature effects and the company Double Negative created additional digital effects.",1.150132
2,"Interstellar (film) Interstellar is a 2014 epic science fiction film co-written, directed, and produced by Christopher Nolan.\nIt stars Matthew McConaughey, Anne Hathaway, Jessica Chastain, Bill Irwin, Ellen Burstyn, Matt Damon, and Michael Caine.\nSet in a dystopian future where humanity is struggling to survive, the film follows a group of astronauts who travel through a wormhole near Saturn in search of a new home for mankind.",1.203064


# Reranking

In [4]:
query = "What is the capital of Canada?"
dense_retrieval_results = dense_retrieval(query, client)


In [9]:
print_result(dense_retrieval_results)


item 0
_additional:{'distance': -150.8031}

lang:en

text:The governor general of the province had designated Kingston as the capital in 1841. However, the major population centres of Toronto and Montreal, as well as the former capital of Lower Canada, Quebec City, all had legislators dissatisfied with Kingston. Anglophone merchants in Quebec were the main group supportive of the Kingston arrangement. In 1842, a vote rejected Kingston as the capital, and study of potential candidates included the then-named Bytown, but that option proved less popular than Toronto or Montreal. In 1843, a report of the Executive Council recommended Montreal as the capital as a more fortifiable location and commercial centre, however, the Governor General refused to execute a move without a parliamentary vote. In 1844, the Queen's acceptance of a parliamentary vote moved the capital to Montreal.

title:Ottawa

url:https://en.wikipedia.org/wiki?curid=22219

views:2000


item 1
_additional:{'distance': -150

## Improving Keyword Search with Rerank

In [ ]:
query_1 = "What is the capital of Canada?"
results = keyword_search(query_1,
                         client,
                         properties=["text", "title", "url", "views", "lang", "_additional {distance}"],
                         num_results=500
                        )

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    # print(result.get('text'))


In [13]:
def rerank_responses(query, responses, num_responses=10):
    reranked_responses = co.rerank(
        model = 'rerank-english-v2.0',
        query = query,
        documents = responses,
        top_n = num_responses,
        )
    return reranked_responses


In [14]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_1, texts)

for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()


i:0

RerankResult<document['text']: Selection of Ottawa as the capital of Canada predates the Confederation of Canada. 
The selection was contentious and not straightforward, with the parliament of the United Province of Canada holding
more than 200 votes over several decades to attempt to settle on a legislative solution to the location of the 
capital., index: 407, relevance_score: 0.9875684>

i:1

RerankResult<document['text']: Montreal was the capital of the Province of Canada from 1844 to 1849, but lost its 
status when a Tory mob burnt down the Parliament building to protest the passage of the Rebellion Losses Bill. 
Thereafter, the capital rotated between Quebec City and Toronto until in 1857, Queen Victoria herself established 
Ottawa as the capital due to strategic reasons. The reasons were twofold. First, because it was located more in the
interior of the Province of Canada, it was less susceptible to attack from the United States. Second, and perhaps 
more importantly, because it lay on the border between French and English Canada, Ottawa was seen as a compromise 
between Montreal, Toronto, Kingston and Quebec City, which were all vying to become the young nation's official 
capital. Ottawa retained the status as capital of Canada when the Province of Canada joined with Nova Scotia and 
New Brunswick to form the Dominion of Canada in 1867., index: 100, relevance_score: 0.9795897>

i:2

...

RerankResult<document['text']: Ottawa is headquarters to numerous major medical organizations and institutions such
as Canadian Red Cross, Canadian Blood Services, Health Canada, Canadian Medical Association, Royal College of 
Physicians and Surgeons of Canada, Canadian Nurses Association, and the Medical Council of Canada., index: 394, 
relevance_score: 0.86153823>

i:8

RerankResult<document['text']: Ontario ( ; ) is one of the thirteen provinces and territories of Canada. Located in
Central Canada, it is Canada's most populous province, with 38.3 percent of the country's population, and is the 
second-largest province by total area (after Quebec). Ontario is Canada's fourth-largest jurisdiction in total area
when the territories of the Northwest Territories and Nunavut are included. It is home to the nation's capital 
city, Ottawa, and the nation's most populous city, Toronto, which is Ontario's provincial capital., index: 228, 
relevance_score: 0.4989891>

i:9

RerankResult<document['text']: With sixty percent of Canada's steel produced in Hamilton by Stelco and Dofasco, the
city has become known as the Steel Capital of Canada. After nearly declaring bankruptcy, Stelco returned to 
profitability in 2004. On August 26, 2007 United States Steel Corporation acquired Stelco for C$38.50 in cash per 
share, owning more than 76 percent of Stelco's outstanding shares. On September 17, 2014, US Steel Canada announced
it was applying for bankruptcy protection and it would close its Hamilton operations., index: 5, relevance_score: 
0.49455282>

## Improving Dense Search with Rerank

In [15]:
query_2 = "Who is the tallest person in history?"
results = dense_retrieval(query_2,client)

for i, result in enumerate(results):
    print(f"i:{i}")
    print(result.get('title'))
    print(result.get('text'))
    print()


i:0

Robert Wadlow

Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton Giant and the Giant of Illinois, 
was a man who was the tallest person in recorded history for whom there is irrefutable evidence. He was born and 
raised in Alton, Illinois, a small city near St. Louis, Missouri.

i:1

Manute Bol

...

Sultan Kösen

Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days early by visiting the Ripley's
Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized statue of Robert Wadlow, the 
tallest man ever at 272 cm (8 ft 11.1 in).

i:4

Netherlands

The Dutch are the tallest people in the world, by nationality, with an average height of for adult males and for 
adult females in 2009. The average height of young males in the Netherlands increased from 5 feet, 4 inches to 
approximately 6 feet between the 1850s until the early 2000s. People in the south are on average about shorter than
those in the north.

In [16]:
texts = [result.get('text') for result in results]
reranked_text = rerank_responses(query_2, texts)
for i, rerank_result in enumerate(reranked_text):
    print(f"i:{i}")
    print(f"{rerank_result}")
    print()


i:0

RerankResult<document['text']: Robert Pershing Wadlow (February 22, 1918 July 15, 1940), also known as the Alton 
Giant and the Giant of Illinois, was a man who was the tallest person in recorded history for whom there is 
irrefutable evidence. He was born and raised in Alton, Illinois, a small city near St. Louis, Missouri., index: 0, 
relevance_score: 0.9734939>

i:1

RerankResult<document['text']: Sultan Kösen (born 10 December 1982) is a Turkish farmer who holds the Guinness 
World Record for tallest living male at . Of Kurdish ethnicity, he is the seventh tallest man in history., index: 
2, relevance_score: 0.8664718>

i:2

RerankResult<document['text']: The Dutch are the tallest people in the world, by nationality, with an average 
height of for adult males and for adult females in 2009. The average height of young males in the Netherlands 
increased from 5 feet, 4 inches to approximately 6 feet between the 1850s until the early 2000s. People in the 
south are on average about shorter than those in the north., index: 4, relevance_score: 0.80162543>

i:3

RerankResult<document['text']: Kösen turned 40 years old on 10 December 2022. He celebrated his birthday a few days
early by visiting the Ripley's Believe It or Not! museum in Orlando, Florida, USA and posing next to a life-sized 
statue of Robert Wadlow, the tallest man ever at 272 cm (8 ft 11.1 in)., index: 3, relevance_score: 0.6874202>

i:4

RerankResult<document['text']: Bol came from a family of extraordinarily tall men and women. He said: "My mother 
was , my father , and my sister is . And my great-grandfather was even taller—." His ethnic group, the Dinka, and 
the Nilotic people of which they are a part, are among the tallest populations in the world. Bol's hometown, 
Turalei, is the origin of other exceptionally tall people, including basketball player Ring Ayuel. "I was born in a
village, where you cannot measure yourself," Bol reflected. "I learned I was 7 foot 7 in 1979, when I was grown. I 
was about 18 or 19.", index: 1, relevance_score: 0.6396235>

# Generating Answers

In [17]:
question = "Are side projects important when you are starting to learn about AI?"

# sample text form Andrew NG newsletter
text = """
The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building exciting careers in this field. A career is a decades-long journey, and the path is not always straightforward. Over many years, I’ve been privileged to see thousands of students as well as engineers in companies large and small navigate careers in AI. In this and the next few letters, I’d like to share a few thoughts that might be useful in charting your own course.

Three key steps of career growth are learning (to gain technical and other skills), working on projects (to deepen skills, build a portfolio, and create impact) and searching for a job. These steps stack on top of each other:

Initially, you focus on gaining foundational technical skills.
After having gained foundational skills, you lean into project work. During this period, you’ll probably keep learning.
Later, you might occasionally carry out a job search. Throughout this process, you’ll probably continue to learn and work on meaningful projects.
These phases apply in a wide range of professions, but AI involves unique elements. For example:

AI is nascent, and many technologies are still evolving. While the foundations of machine learning and deep learning are maturing — and coursework is an efficient way to master them — beyond these foundations, keeping up-to-date with changing technology is more important in AI than fields that are more mature.
Project work often means working with stakeholders who lack expertise in AI. This can make it challenging to find a suitable project, estimate the project’s timeline and return on investment, and set expectations. In addition, the highly iterative nature of AI projects leads to special challenges in project management: How can you come up with a plan for building a system when you don’t know in advance how long it will take to achieve the target accuracy? Even after the system has hit the target, further iteration may be necessary to address post-deployment drift.
While searching for a job in AI can be similar to searching for a job in other sectors, there are some differences. Many companies are still trying to figure out which AI skills they need and how to hire people who have them. Things you’ve worked on may be significantly different than anything your interviewer has seen, and you’re more likely to have to educate potential employers about some elements of your work.
Throughout these steps, a supportive community is a big help. Having a group of friends and allies who can help you — and whom you strive to help — makes the path easier. This is true whether you’re taking your first steps or you’ve been on the journey for years.

I’m excited to work with all of you to grow the global AI community, and that includes helping everyone in our community develop their careers. I’ll dive more deeply into these topics in the next few weeks.

Last week, I wrote about key steps for building a career in AI: learning technical skills, doing project work, and searching for a job, all of which is supported by being part of a community. In this letter, I’d like to dive more deeply into the first step.

More papers have been published on AI than any person can read in a lifetime. So, in your efforts to learn, it’s critical to prioritize topic selection. I believe the most important topics for a technical career in machine learning are:

Foundational machine learning skills. For example, it’s important to understand models such as linear regression, logistic regression, neural networks, decision trees, clustering, and anomaly detection. Beyond specific models, it’s even more important to understand the core concepts behind how and why machine learning works, such as bias/variance, cost functions, regularization, optimization algorithms, and error analysis.
Deep learning. This has become such a large fraction of machine learning that it’s hard to excel in the field without some understanding of it! It’s valuable to know the basics of neural networks, practical skills for making them work (such as hyperparameter tuning), convolutional networks, sequence models, and transformers.
Math relevant to machine learning. Key areas include linear algebra (vectors, matrices, and various manipulations of them) as well as probability and statistics (including discrete and continuous probability, standard probability distributions, basic rules such as independence and Bayes rule, and hypothesis testing). In addition, exploratory data analysis (EDA) — using visualizations and other methods to systematically explore a dataset — is an underrated skill. I’ve found EDA particularly useful in data-centric AI development, where analyzing errors and gaining insights can really help drive progress! Finally, a basic intuitive understanding of calculus will also help. In a previous letter, I described how the math needed to do machine learning well has been changing. For instance, although some tasks require calculus, improved automatic differentiation software makes it possible to invent and implement new neural network architectures without doing any calculus. This was almost impossible a decade ago.
Software development. While you can get a job and make huge contributions with only machine learning modeling skills, your job opportunities will increase if you can also write good software to implement complex AI systems. These skills include programming fundamentals, data structures (especially those that relate to machine learning, such as data frames), algorithms (including those related to databases and data manipulation), software design, familiarity with Python, and familiarity with key libraries such as TensorFlow or PyTorch, and scikit-learn.
This is a lot to learn! Even after you master everything in this list, I hope you’ll keep learning and continue to deepen your technical knowledge. I’ve known many machine learning engineers who benefitted from deeper skills in an application area such as natural language processing or computer vision, or in a technology area such as probabilistic graphical models or building scalable software systems.

How do you gain these skills? There’s a lot of good content on the internet, and in theory reading dozens of web pages could work. But when the goal is deep understanding, reading disjointed web pages is inefficient because they tend to repeat each other, use inconsistent terminology (which slows you down), vary in quality, and leave gaps. That’s why a good course — in which a body of material has been organized into a coherent and logical form — is often the most time-efficient way to master a meaningful body of knowledge. When you’ve absorbed the knowledge available in courses, you can switch over to research papers and other resources.

Finally, keep in mind that no one can cram everything they need to know over a weekend or even a month. Everyone I know who’s great at machine learning is a lifelong learner. In fact, given how quickly our field is changing, there’s little choice but to keep learning if you want to keep up. How can you maintain a steady pace of learning for years? I’ve written about the value of habits. If you cultivate the habit of learning a little bit every week, you can make significant progress with what feels like less effort.

In the last two letters, I wrote about developing a career in AI and shared tips for gaining technical skills. This time, I’d like to discuss an important step in building a career: project work.

It goes without saying that we should only work on projects that are responsible and ethical, and that benefit people. But those limits leave a large variety to choose from. I wrote previously about how to identify and scope AI projects. This and next week’s letter have a different emphasis: picking and executing projects with an eye toward career development.

A fruitful career will include many projects, hopefully growing in scope, complexity, and impact over time. Thus, it is fine to start small. Use early projects to learn and gradually step up to bigger projects as your skills grow.

When you’re starting out, don’t expect others to hand great ideas or resources to you on a platter. Many people start by working on small projects in their spare time. With initial successes — even small ones — under your belt, your growing skills increase your ability to come up with better ideas, and it becomes easier to persuade others to help you step up to bigger projects.

What if you don’t have any project ideas? Here are a few ways to generate them:

Join existing projects. If you find someone else with an idea, ask to join their project.
Keep reading and talking to people. I come up with new ideas whenever I spend a lot of time reading, taking courses, or talking with domain experts. I’m confident that you will, too.
Focus on an application area. Many researchers are trying to advance basic AI technology — say, by inventing the next generation of transformers or further scaling up language models — so, while this is an exciting direction, it is hard. But the variety of applications to which machine learning has not yet been applied is vast! I’m fortunate to have been able to apply neural networks to everything from autonomous helicopter flight to online advertising, partly because I jumped in when relatively few people were working on those applications. If your company or school cares about a particular application, explore the possibilities for machine learning. That can give you a first look at a potentially creative application — one where you can do unique work — that no one else has done yet.
Develop a side hustle. Even if you have a full-time job, a fun project that may or may not develop into something bigger can stir the creative juices and strengthen bonds with collaborators. When I was a full-time professor, working on online education wasn’t part of my “job” (which was doing research and teaching classes). It was a fun hobby that I often worked on out of passion for education. My early experiences recording videos at home helped me later in working on online education in a more substantive way. Silicon Valley abounds with stories of startups that started as side projects. So long as it doesn’t create a conflict with your employer, these projects can be a stepping stone to something significant.
Given a few project ideas, which one should you jump into? Here’s a quick checklist of factors to consider:

Will the project help you grow technically? Ideally, it should be challenging enough to stretch your skills but not so hard that you have little chance of success. This will put you on a path toward mastering ever-greater technical complexity.
Do you have good teammates to work with? If not, are there people you can discuss things with? We learn a lot from the people around us, and good collaborators will have a huge impact on your growth.
Can it be a stepping stone? If the project is successful, will its technical complexity and/or business impact make it a meaningful stepping stone to larger projects? (If the project is bigger than those you’ve worked on before, there’s a good chance it could be such a stepping stone.)
Finally, avoid analysis paralysis. It doesn’t make sense to spend a month deciding whether to work on a project that would take a week to complete. You'll work on multiple projects over the course of your career, so you’ll have ample opportunity to refine your thinking on what’s worthwhile. Given the huge number of possible AI projects, rather than the conventional “ready, aim, fire” approach, you can accelerate your progress with “ready, fire, aim.”

"""


In [18]:
import warnings
warnings.filterwarnings('ignore')


## Chunking

In [19]:
# chunking
# Split into a list of paragraphs
texts = text.split('\n\n')

# Clean up to remove empty spaces and new lines
texts = np.array([t.strip(' \n') for t in texts if t])


In [20]:
print(texts[0:2])


['The rapid rise of AI has led to a rapid rise in AI jobs, and many people are building exciting careers in this 
field. A career is a decades-long journey, and the path is not always straightforward. Over many years, I’ve been 
privileged to see thousands of students as well as engineers in companies large and small navigate careers in AI. 
In this and the next few letters, I’d like to share a few thoughts that might be useful in charting your own 
course.'
 'Three key steps of career growth are learning (to gain technical and other skills), working on projects (to 
deepen skills, build a portfolio, and create impact) and searching for a job. These steps stack on top of each 
other:']

## embeddings

In [23]:
# Get the embeddings
response = co.embed(
    texts=texts.tolist(),
).embeddings

# Check the dimensions of the embeddings
embeds = np.array(response)


default model on embed will be deprecated in the future, please specify a model in the request.


In [24]:
# create a search index after getting the embeddings
search_index = AnnoyIndex(embeds.shape[1], 'angular')
# Add all the vectors to the search index
for i in range(len(embeds)):
    search_index.add_item(i, embeds[i])
search_index.build(10) # 10 trees
search_index.save('test_andrewNG.ann')


True

In [25]:
# search articles

def search_andrews_article(query):
    # Get the query's embedding
    query_embed = co.embed(texts=[query]).embeddings

    # Retrieve the nearest neighbors
    similar_item_ids = search_index.get_nns_by_vector(query_embed[0],
                                                    10,
                                                  include_distances=True)

    search_results = texts[similar_item_ids[0]]

    return search_results


In [26]:
results = search_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?"
)

print(results[0])


default model on embed will be deprecated in the future, please specify a model in the request.


Join existing projects. If you find someone else with an idea, ask to join their project.
Keep reading and talking to people. I come up with new ideas whenever I spend a lot of time reading, taking 
courses, or talking with domain experts. I’m confident that you will, too.
Focus on an application area. Many researchers are trying to advance basic AI technology — say, by inventing the 
next generation of transformers or further scaling up language models — so, while this is an exciting direction, it
is hard. But the variety of applications to which machine learning has not yet been applied is vast! I’m fortunate 
to have been able to apply neural networks to everything from autonomous helicopter flight to online advertising, 
partly because I jumped in when relatively few people were working on those applications. If your company or school
cares about a particular application, explore the possibilities for machine learning. That can give you a first 
look at a potentially creative application — one where you can do unique work — that no one else has done yet.
Develop a side hustle. Even if you have a full-time job, a fun project that may or may not develop into something 
bigger can stir the creative juices and strengthen bonds with collaborators. When I was a full-time professor, 
working on online education wasn’t part of my “job” (which was doing research and teaching classes). It was a fun 
hobby that I often worked on out of passion for education. My early experiences recording videos at home helped me 
later in working on online education in a more substantive way. Silicon Valley abounds with stories of startups 
that started as side projects. So long as it doesn’t create a conflict with your employer, these projects can be a 
stepping stone to something significant.
Given a few project ideas, which one should you jump into? Here’s a quick checklist of factors to consider:

In [27]:
# Generating Answers

def ask_andrews_article(question, num_generations=1):

    # Search the text archive
    results = search_andrews_article(question)

    # Get the top result
    context = results[0]

    # Prepare the prompt
    prompt = f"""
    Excerpt from the article titled "How to Build a Career in AI"
    by Andrew Ng:
    {context}
    Question: {question}

    Extract the answer of the question from the text provided.
    If the text doesn't contain the answer,
    reply that the answer is not available."""

    prediction = co.generate(
        prompt=prompt,
        max_tokens=70,
        model="command-nightly",
        temperature=0.5,
        num_generations=num_generations
    )

    return prediction.generations


In [28]:
results = ask_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?",

)

print(results[0])


default model on embed will be deprecated in the future, please specify a model in the request.


Yes, side projects are a good idea when trying to build a career in AI as it allows you to strengthen bonds with 
collaborators and stir up creativity. Having a side hustle will also allow you to apply AI to a variety of 
different applications and explore new possibilities with machine learning.

In [29]:
results = ask_andrews_article(
    "Are side projects a good idea when trying to build a career in AI?",
    num_generations=3
)

for gen in results:
    print(gen)
    print('--')


default model on embed will be deprecated in the future, please specify a model in the request.


Yes, side projects are a good idea when trying to build a career in AI as it allows you to strengthen your skills 
and knowledge in the field, work on an exciting idea, and establish meaningful connections with other professionals
in the industry. It also provides the potential for future growth and opportunity, and Chancing upon new ideas and 
applications that may not have

--

Yes, side projects are a good idea when trying to build a career in AI as it allows you to strengthen your skills 
and knowledge in the field, and can even lead to new opportunities and collaborations.

--

Yes, side projects are a good idea when trying to build a career in AI as it allows you to strengthen bonds with 
collaborators and stir up creativity. Having a side hustle project, specifically in AI, can provide you with a 
competitive advantage in building a career in AI as it allows you to practice and apply the skills you have learned
in a hands-

--

In [30]:
results = ask_andrews_article(
    "What is the most viewed televised event?",
    num_generations=5
)

for gen in results:
    print(gen)
    print('--')


default model on embed will be deprecated in the future, please specify a model in the request.


The answer to the question "What is the most viewed televised event?" is not available in the provided text.

--

The answer to the question "What is the most viewed televised event?" is not available in the provided text.

--

...

--

The answer to the question "What is the most viewed televised event?" is not available in the provided text.

--

The question you asked is not included in the excerpt from Andrew Ng's article that you provided; hence, I do not 
have an answer for you. 

I can provide answers to any other questions you may have regarding the excerpt, or any other safe and appropriate 
queries you propose.

--